<a href="https://colab.research.google.com/github/Sammcfadz/Geog761-Project-WewillaskAI/blob/main/sallylandslideslayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

setting up this thing

In [ ]:
# Set up GEE API
import ee
ee.Authenticate()
ee.Initialize(project='lab-1-761')

still setting up

In [ ]:
# Our usual set up routine
!pip install geemap --quiet

import geemap
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans

#<- Remember to change this to your own project's name!

In [ ]:
import geemap

tifffile of landslide polygons cut to auckland

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

To upload assets using the Earth Engine command-line tool, you'll need to have it installed and authenticated.

1.  **Install the Earth Engine command-line tool:**
    You can install it using pip:

In [ ]:
# Define area of interest (e.g., Auckland, NZ)
# Adjusted coordinates to cover a wider area of Auckland and a little bit more south
# TODO: Please verify and adjust these coordinates to define your desired AOI.
aoi = ee.Geometry.Rectangle([174.1, -37.3, 175.5, -36.0]) # Adjusted southern coordinate
Map = geemap.Map(center=[-36.85, 174.75], zoom=9) # Centered on Auckland
Map.addLayer(aoi, {}, 'AOI')

# Visualize on the map to check got it right... (always a good idea)
Map

In [ ]:
import ee

# Make sure Earth Engine is initialized with your project
# If not, run cell c9JFCuTacUF6 after replacing 'lab-1-761' with your project ID

# Load the Earth Engine asset
map4_image = ee.Image('projects/lab-1-761/assets/Map4')

# You can now use the 'map4_image' variable in your Earth Engine computations and visualizations
# For example, to print information about the image:
print('Map4 Image Info:', map4_image.getInfo())

In [ ]:
# Set up your filter Sentinel-2 imagery
# A function that masks clouds in your S2 images via QA band
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
           qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000).select(['B2', 'B3', 'B4', 'B8'])

# Now build the clean collection with selected bands
s2_collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
      .filterBounds(aoi)
      .filterDate('2023-02-01', '2023-03-20')
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
      .map(mask_s2_clouds))

# Check the number of images in the collection
print('Number of images in the collection:', s2_collection.size().getInfo())

s2 = s2_collection.median().clip(aoi) #<- think about this step and what it is doing to your image when you build your single composite from the image collection

# Print the min and max pixel values to help with visualization
print('Min pixel value:', s2.reduceRegion(reducer=ee.Reducer.min(), geometry=aoi, scale=1000, maxPixels=1e9).getInfo())
print('Max pixel value:', s2.reduceRegion(reducer=ee.Reducer.max(), geometry=aoi, scale=1000, maxPixels=1e9).getInfo())

# Print image information
print('Image information:', s2.getInfo())


bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
band_names = {
    'B2': 'Blue (0.45–0.52 μm)',
    'B3': 'Green (0.52–0.60 μm)',
    'B4': 'Red (0.63–0.69 μm)',
    'B8': 'NIR (0.77–0.90 μm)',
    'B11': 'SWIR1 (1.55–1.75 μm)',
    'B12': 'SWIR2 (2.09–2.35 μm)'
}
scale_factor = 0.0001

# Create a dictionary for visualization parameters
vis_params = {
    'bands': ['B4', 'B3', 'B2'],  # Using B4, B3, B2 for true color composite
    'min': 0.0, # Adjusted min
    'max': 0.5,  # Adjusted max based on common scaled values
    'gamma': 1.9
}


Map = geemap.Map(center=[-36.85, 174.75], zoom=9) #<- reset the map object to clear out other layers from earlier cells, centered on Auckland
Map.addLayer(s2, vis_params, 'hello')
Map.addLayer(aoi, {'color': 'red'}, 'AOI')
Map.centerObject(aoi, zoom=9)
Map

remove clouds from image for processing

In [ ]:

AOI = ee.Geometry.Point(174.7633, -36.8485)
START_DATE = '2023-02-01'
END_DATE = '2023-03-20'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.14
CLD_PRJ_DIST = 1
BUFFER = 5


2nd step of cloud removing



In [ ]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))



s2_sr_cld_col_eval = get_s2_sr_cld_col(aoi, START_DATE, END_DATE)

In [ ]:
Map = geemap.Map()
Map.centerObject(aoi, 10)
# Take the median of the collection and clip it to the AOI
clipped_s2_image = s2_sr_cld_col_eval.median().clip(aoi)
Map.addLayer(clipped_s2_image, {'bands':['B4','B3','B2'], 'min':0, 'max':3000}, 'S2 collection (clipped)')
Map.addLayer(aoi, {'color': 'red'}, 'AOI')

Map

3rd step of cloud processing

In [ ]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))




def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))



def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

4th step of cloud processing

In [ ]:
def display_cloud_layers_geemap(col, AOI):
    # Mosaic the image collection.
    img = col.mosaic()

    # Clip the mosaicked image to the AOI
    clipped_img = img.clip(aoi)


    # Subset layers and prepare them for display.
    clouds = clipped_img.select('clouds').selfMask()
    shadows = clipped_img.select('shadows').selfMask()
    dark_pixels = clipped_img.select('dark_pixels').selfMask()
    probability = clipped_img.select('probability')
    cloudmask = clipped_img.select('cloudmask').selfMask()
    cloud_transform = clipped_img.select('cloud_transform')

    # Create a geemap Map object centered on AOI.
    m = geemap.Map()
    m.centerObject(AOI, 12)

    # Add layers
    m.addLayer(clipped_img, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1}, 'S2 image (clipped)')
    m.addLayer(probability, {'min': 0, 'max': 100}, 'probability (cloud)')
    m.addLayer(clouds, {'palette': ['e056fd']}, 'clouds')
    m.addLayer(cloud_transform, {'min': 0, 'max': 1, 'palette': ['white', 'black']}, 'cloud_transform')
    m.addLayer(dark_pixels, {'palette': ['orange']}, 'dark_pixels')
    m.addLayer(shadows, {'palette': ['yellow']}, 'shadows')
    m.addLayer(cloudmask, {'palette': ['orange']}, 'cloudmask')

    return m



s2_sr_cld_col_eval_disp = s2_sr_cld_col_eval.map(add_cld_shdw_mask)
display_map = display_cloud_layers_geemap(s2_sr_cld_col_eval_disp, AOI)
display_map

next part here we go

In [ ]:
def apply_cld_shdw_mask(img):
    cloudmask = img.select('cloudmask')
    # Mask cloudy pixels
    masked_img = img.updateMask(cloudmask.Not())
    # Keep cloudmask and probability bands
    cloudmask_band = img.select('cloudmask')
    probability_band = img.select('probability')
    masked_img = masked_img.addBands([cloudmask_band, probability_band], overwrite=True)
    return masked_img

s2_sr_cld_col = get_s2_sr_cld_col(aoi, START_DATE, END_DATE)

s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())




def display_cloud_free_mosaic(s2_sr_median, AOI):
    # Create a geemap Map object and center it on the AOI
    m = geemap.Map()
    m.centerObject(AOI, 12)

    # Clip the image to the AOI before adding it to the map
    clipped_s2_sr_median = s2_sr_median.clip(aoi)

    # Add the Sentinel-2 RGB mosaic
    m.addLayer(
        clipped_s2_sr_median,
        {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.5},
        'S2 cloud-free mosaic')




    # Add the Map4 image asset as a layer, applying selfMask to remove the black background
    if 'map4_image' in globals() and isinstance(map4_image, ee.Image):
        # You might need to adjust visualization parameters for Map4 image
        # Replace {'bands': ['b1'], 'min': 0, 'max': 255} with appropriate vis_params for your Map4 data
        m.addLayer(map4_image.selfMask(), {'bands': ['b4', 'b3', 'b2'], 'min': 0, 'max': 255}, 'Map4 Asset') # Added placeholder vis_params
    else:
        print("Warning: 'map4_image' not found or is not an Earth Engine image. Cannot add Map4 layer.")

    return m

display_map = display_cloud_free_mosaic(s2_sr_median, aoi)
display_map

export landsldie layer